In [1]:
#Importamos las librerias necesarias para el análisis de datos 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
#Importamos plotly para la visualización de los datos
import plotly.express as px
import plotly.graph_objects as go

In [2]:
df = pd.read_csv(r'C:\Users\spide\Desktop\Bootcamp\mi_entorno\Modulo_3\proyecto_final_modulo_3\vehiculos-de-segunda-mano-sample.csv')

In [3]:
df.shape

(100000, 28)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 28 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   vehicle_type            100000 non-null  object 
 1   make                    100000 non-null  object 
 2   model                   100000 non-null  object 
 3   version                 99842 non-null   object 
 4   fuel                    96810 non-null   object 
 5   year                    100000 non-null  int64  
 6   kms                     99189 non-null   float64
 7   power                   99709 non-null   float64
 8   doors                   0 non-null       float64
 9   shift                   98982 non-null   object 
 10  color                   0 non-null       float64
 11  photos                  100000 non-null  int64  
 12  description             55810 non-null   object 
 13  price                   100000 non-null  int64  
 14  currency             

## Valores duplicados

In [5]:
df.duplicated().sum() #filas duplicadas

237

In [6]:
df = df.drop_duplicates()

## Valores nulos

In [15]:
df.isnull().sum()

vehicle_type                  0
make                          0
model                         0
version                     158
fuel                       3177
year                          0
kms                         809
power                       283
doors                     99763
shift                      1018
color                     99763
photos                        0
description               44014
price                         0
currency                      0
location                      0
publish_date                  0
update_date                   0
dealer_name                  92
dealer_description        18345
dealer_address               92
dealer_zip_code              92
dealer_city                  92
dealer_country_code           0
dealer_is_professional        0
dealer_website            19997
dealer_registered_at         92
date                          0
dtype: int64

In [7]:
((df.isnull().sum())/df.shape[0])*100

vehicle_type                0.000000
make                        0.000000
model                       0.000000
version                     0.158375
fuel                        3.184547
year                        0.000000
kms                         0.810922
power                       0.283672
doors                     100.000000
shift                       1.020418
color                     100.000000
photos                      0.000000
description                44.118561
price                       0.000000
currency                    0.000000
location                    0.000000
publish_date                0.000000
update_date                 0.000000
dealer_name                 0.092219
dealer_description         18.388581
dealer_address              0.092219
dealer_zip_code             0.092219
dealer_city                 0.092219
dealer_country_code         0.000000
dealer_is_professional      0.000000
dealer_website             20.044505
dealer_registered_at        0.092219
d

In [20]:
# Ajustar las opciones de visualización de pandas
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# Mostrar el resultado completo
df[df['fuel'].isnull()][['make','model']].value_counts()

make           model             
Toyota         C-HR                  489
Mazda          2                     128
Toyota         Corolla               118
CUPRA          Leon                  117
Lexus          UX                    110
Ford           Puma                  109
Lexus          ES                     98
Toyota         Yaris                  90
               Auris                  80
KIA            Niro                   79
Fiat           500                    79
Lexus          NX                     70
KIA            Ceed                   68
               Sportage               67
Hyundai        IONIQ                  65
Alfa Romeo     Tonale                 64
Honda          Insight                56
BMW            X7                     50
Renault        Arkana                 50
Lexus          RC                     48
Citroen        C5 X                   44
               C3                     41
Lexus          RX                     37
BMW            Serie 4 

In [14]:
df.select_dtypes(include=['object']).nunique()

vehicle_type               1
make                      66
model                    748
version                 8658
fuel                       4
shift                      2
description             6439
currency                   1
location                 669
publish_date             424
update_date               38
dealer_name             1678
dealer_description      1181
dealer_address          1675
dealer_city              665
dealer_country_code        1
dealer_website           888
dealer_registered_at    1177
date                      13
dtype: int64

In [9]:
df['photos'].unique()

array([10,  8,  6,  2,  9,  5,  1,  7,  4,  3], dtype=int64)

In [10]:
df["dealer_website"][0]

'www.cocheauto.com'

In [11]:
df.columns

Index(['vehicle_type', 'make', 'model', 'version', 'fuel', 'year', 'kms',
       'power', 'doors', 'shift', 'color', 'photos', 'description', 'price',
       'currency', 'location', 'publish_date', 'update_date', 'dealer_name',
       'dealer_description', 'dealer_address', 'dealer_zip_code',
       'dealer_city', 'dealer_country_code', 'dealer_is_professional',
       'dealer_website', 'dealer_registered_at', 'date'],
      dtype='object')